# Julia Fundamentals

> _Walks like Python, runs like C_

- Open-source language from MIT
- Features
  - Dynamically-typed
  - Fast (thanks to JIT)
  - Composable (i.e. multiple-dispatch)
  - LISP (metaprogramming capabilities)
  - Reproducible

## Types

In [ ]:
3 + 2

In [ ]:
3 + 2.0

In [ ]:
typeof(3)

In [ ]:
typeof(2.0)

In [ ]:
typeof([1,2,3])

Here, `Array` is a **parametric type**. The type parameters are
* `Int64` (element type)
* `1` (dimensionality)

Hence `Array{Int64, 1}` means that we have a vector that can hold 64-bit integers.

In [ ]:
eltype([1,2,3])

In [ ]:
typeof(rand(3,3))

In [ ]:
typeof(5:7)

In [ ]:
3 isa Int64

In [ ]:
3 isa Float64

In [ ]:
3 isa Number

**Concrete types**
* types of values ("objects")
* specify data structure
* Example: `Float64`

**Abstract types**
* cannot be instantiated
* their descendant types share "behavior" ([duck typing](https://en.wikipedia.org/wiki/Duck_typing))
* Examples:

The abstract type `Number` indicates that one can do number-like things, e.g. `+`,`-`,`*`, and `/`, with corresponding values. In this category we have (concrete) things like `Float64` (double precision float) and `Int32` (single precision integer) numbers.

An `AbstractArray` is a type that, e.g., allows indexing `A[i]`. Examples include regular arrays (`Array`), as well as ranges (`UnitRange`).

### Inspecting the type tree

In [ ]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

In [ ]:
print_tree(Number) # print a branch of the type tree

Everything is a subtype of `Any`. It's the root of the type tree.

Note that **concrete types are the leaves of the type tree** whereas **abstract types are nodes** in the type graph.

In [ ]:
1 isa Integer

In [ ]:
Integer <: Number

In [ ]:
supertype(Integer)

## Functions

Julia is based on the multiple-dispatch paradigm (aka multimethods or free functions).
What this means is that functions can have several different implementations, called **methods**.
With multiple-dispatch, when calling a function, the method to run is selected **based on the types of all the passed arguments**.

Let's define a *function* that calculates the absolute value of a number (like the built-in `abs` already does).

**How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?**

* Real number
  * "Drop the sign."
    * => `myabs(-4.32) = 4.32`
* Complex number:
  * "Square root of z times the complex conjugate of z."
    * => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [ ]:
myabs(x::Float64) = sign(x) * x

In [ ]:
myabs(-4.32)

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

In [ ]:
myabs(1.0 + 1.0im)

But what if I use an `Int`? How can I generalize the method to more types?

In [ ]:
myabs(-1)

In [ ]:
myabs(x::Real) = sign(x) * x

In [ ]:
myabs(-1)

And what about `Complex` number with another precision or other parametric type?

In [ ]:
myabs(3 + 4im)

In [ ]:
myabs(z::Complex) = sqrt(real(z * conj(z)))

In [ ]:
myabs(3 + 4im)

Check out that here `Complex` is neither an abstract type nor a concrete type. It'a a `UnionAll`: it represents the set of all `Complex{T}` for all `T`.

In [ ]:
Complex{T} where {T}

In [ ]:
Complex{T} where {T<:Int}

In [ ]:
Complex{T} where {T<:String}

### Method introspection

In [ ]:
methods(myabs)

In [ ]:
@which myabs(-4.32)

In [ ]:
@which myabs(1.0 + 1.0im)

Unlike other strongly typed languages (e.g. C/C++/Fortran), type annotations in function signatures **virtually never affect performance**!
One should therefore generally make them as generic as possible but as specific as necessary.

In [ ]:
methods(myabs, (Real,))

In [ ]:
collect(Base.specializations(methods(myabs, (Real,))[2]))

In [ ]:
myabs(Int32(1))

In [ ]:
collect(Base.specializations(methods(myabs, (Real,))[2]))

### Multiple dispatch

Which method gets executed when you call a generic function `f` for a given set of input arguments?

**Answer:** Julia always chooses the **most specific method** by considering **all input argument types**.

(Since methods belong to generic functions rather than objects no function argument is special.)

In [ ]:
f(a, b)                   = "a and b are anything"
f(a::Number, b)           = "a is a number, b is anything"
f(a, b::Number)           = "a is anything, b is a number"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::AbstractFloat, b::AbstractFloat) = "a and b are both floats"
f(a::Float32, b::Float32) = "a and b are both 32-bit floats"

In [ ]:
methods(f)

In [ ]:
f("Hello", "Julia")

In [ ]:
f(1.5, 2)

In [ ]:
f(1, "Julia")

In [ ]:
f(1, 2)

In [ ]:
f(1.2, 3.4)

In [ ]:
f(1.2f0, 3.4f0) # 1.2f0 is the literal way to write Float32(1.2)

In [ ]:
@which f(1.2, 3.4)

In [ ]:
@which f(1, "UCL")

### Built-in Julia function

**Julia's built-in functions are not special by any means.**

In [ ]:
methods(+)

In [ ]:
@which true + false

In [ ]:
@which "Hello"*"World!"

We can easily modify or add methods to them as well.

In [ ]:
import Base: + # explicitly import functions to override/add methods

+(x::String, y::String) = x * " and " * y

# or the equivalent without importing:
# Base.:+(x::String, y::String) = x * " and " * y

In [ ]:
"A" + "B"